## SDS 21 - Pneumonia detection of X-ray imaging using CNN

Steps in Implementing CNN
- Import required libraries and packages
- Test Data path -> chest_xray\test\Normal,Pneumonia
- Train Data Path -> chest_xray\train\Normal,Pneumonia
- Validation Data Path -> chest_xray\val\Normal,Pneumonia
- Set data locations
- Labelling dataset (normal or pneumonia)
- Count the number of images per folder to see if the dataset is balanced or not
- Data Augmentation to handle the imbalance
- Oversampling minority class (Handle Imbalance)
- Compute class weights to handle imbalance
- CNN Architecture, Build layers
- Compile model with adam optimizer
- Define callbacks to setup checkpoints or have early stops
- Train model
- Evaluate performance on test set using Precision, Recall and F1-score
- Save the model

In [70]:
#Import libraries and packages
# We are going to use Tensorflow, scikit learn for metrics
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report,precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight

In [71]:
#Set Data locations
#Data is moved out of the solution folder and the detailed paths are provided, instead of short path
train_path = r'C:\\Users\\ghk23\OneDrive\\XPS_OneDrive\\Databases\\chest_xray\\train'
test_path = 'C:/Users/ghk23/OneDrive/XPS_OneDrive/Databases/chest_xray/test'
val_path = r'C:/Users/ghk23/OneDrive/XPS_OneDrive/Databases/chest_xray/val'
data_gen = ImageDataGenerator(rescale=1./255)

In [72]:
# Labelling Dataset - Set class indices
#We need to tell the model the classes and the binary options avaiable.
class_indices = train_data.class_indices  # {'NORMAL': 0, 'PNEUMONIA': 1}
normal_idx = class_indices['NORMAL']
pneumonia_idx = class_indices['PNEUMONIA']

# Per Category, count the number of images.
num_normal = sum(1 for _, label in zip(train_data.filepaths, train_data.classes) if label == normal_idx)
num_pneumonia = sum(1 for _, label in zip(train_data.filepaths, train_data.classes) if label == pneumonia_idx)

print(f"\nTrain Data Image Count Summary\nOriginal count: NORMAL = {num_normal}, PNEUMONIA = {num_pneumonia}")


Train Data Image Count Summary
Original count: NORMAL = 1341, PNEUMONIA = 3875


This shows that the dataset is imbalanced. 
Now, we need to implement certain balancing measures.
We are going to use three methods to achieve the same:
- Data Augmentation (Train Dataset)
- By Oversampling minority class
- Compute class weights to handle imbalance dataset - OPTIONAL (With Class Weights, Precision was 78%, Recall was 99% and F1-score was 86%. Based on the result when removed, it may or may not be included in the end)

In [73]:
# Data augmentation to handle class imbalance and improve generalization
#Data Augmentation is one of the simplest and easiest ways to reduce the imbalance.
data_gen = ImageDataGenerator(
    rescale=1./255,      # Normalize pixel values to [0,1] - Converting image data to pixel data
    rotation_range=30,   # Rotate images by up to 30 degrees
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.2,  # Zoom in and out by 10%
    brightness_range=[0.8, 1.2],  # Adjust brightness between 80% and 120%
    fill_mode='nearest' #Fill missing pixels
)

# Load training data with augmentations
train_data = data_gen.flow_from_directory(
    train_path,
    target_size=(224, 224), #Used sharper images than the 150 ones to make the artifacts more pronounced
    batch_size=32,
    class_mode='binary', #Binary class mode as we have only normal and pneumonia classes
    color_mode='grayscale', #Images are grayscale and not RGB
    shuffle=True) #Shuffle the images - Default is true

# Load testing data without augmentation (only rescaling)
test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=False)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


- We will first identify based on our indexes which one is the minority class.
- Next, we will obtain the number of samples to unsample. This is nothing but the delta between the number of normal images and
- The number of Pneumonia images. So, the different will be our target number to unsample.
- For the minority class, we will generate these many images in memory.
- Oversampling minority class is the best way to handle imbalance than the undersampling majority class. 
- The reason for this is to avoid underfitting and loss of samples.

In [74]:
# Handling class imbalance by oversampling minority class
if num_normal > num_pneumonia:
    minority_class = pneumonia_idx
    majority_class = normal_idx
else:
    minority_class = normal_idx
    majority_class = pneumonia_idx

# Identify the number of samples to upsample
num_to_upsample = abs(num_normal - num_pneumonia)

# Generate additional images for the minority class
minority_images = [train_data.filepaths[i] for i in range(len(train_data.filepaths)) if train_data.classes[i] == minority_class]
upsampled_images = resample(minority_images, replace=True, n_samples=num_to_upsample, random_state=42)

print(f"Oversampling done: Added {len(upsampled_images)} images to balance the dataset")

Oversampling done: Added 2534 images to balance the dataset


In [75]:
#Recount the images and print (To check if we did the oversampling in the right way)
num_normal_updated = num_normal + (num_to_upsample if minority_class == normal_idx else 0)
num_pneumonia_updated = num_pneumonia + (num_to_upsample if minority_class == pneumonia_idx else 0)

print(f"Updated count: NORMAL = {num_normal_updated}, PNEUMONIA = {num_pneumonia_updated}")

Updated count: NORMAL = 3875, PNEUMONIA = 3875


In [76]:
#Initially implemented, but may not be needed
#Compute Class weights to handle imbalance
# class_weights = compute_class_weight('balanced', classes=np.unique(train_data.classes), y=train_data.classes)
# class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
# print(f"Computed class weights: {class_weight_dict}")

### CNN Architecture, Model Building, Training, Testing

In [77]:
# Build CNN model with multiple convolutional and pooling layers
#Here a 50% dropout number is provided. This turns-off 50% of the neurons and thereby improves the performance. 
#It reduces the dependency on a few weights and reduces the chance of overfitting
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])

In [78]:
# Compile model with Adam optimizer and binary cross-entropy loss function
# Adam sets the learning rate. Slow learning rate like 0.0001 will converge the output slowly. This is good for accuracy.
# As this is a binary classification probel, binary cross entropy is used and accuracy is used to show it in the output.
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [81]:
# Define callbacks to save best model and stop early if no improvement
# Set intermediatry checkpoints (ONly if it is the best model)
# So, model will stop after 5 epochs with no improvement.
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [82]:
# Train model using the training dataset with validation monitoring
history = model.fit(train_data, validation_data=test_data, epochs=10, class_weight=class_weight_dict, callbacks=[checkpoint, early_stop])

Epoch 1/10
163/163 [==============================] - ETA: 0s - loss: 0.2852 - accuracy: 0.8742

C:\Users\ghk23\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


163/163 [==============================] - 220s 1s/step - loss: 0.2852 - accuracy: 0.8742 - val_loss: 0.2960 - val_accuracy: 0.8654
Epoch 2/10
163/163 [==============================] - 231s 1s/step - loss: 0.2672 - accuracy: 0.8848 - val_loss: 0.2661 - val_accuracy: 0.8926
Epoch 3/10
163/163 [==============================] - 234s 1s/step - loss: 0.2371 - accuracy: 0.9053 - val_loss: 0.2540 - val_accuracy: 0.9022
Epoch 4/10
163/163 [==============================] - 230s 1s/step - loss: 0.2257 - accuracy: 0.9072 - val_loss: 0.2489 - val_accuracy: 0.9087
Epoch 5/10
163/163 [==============================] - 230s 1s/step - loss: 0.2072 - accuracy: 0.9160 - val_loss: 0.2568 - val_accuracy: 0.8990
Epoch 6/10
163/163 [==============================] - 232s 1s/step - loss: 0.1909 - accuracy: 0.9233 - val_loss: 0.2584 - val_accuracy: 0.8942
Epoch 7/10
163/163 [==============================] - 231s 1s/step - loss: 0.1855 - accuracy: 0.9254 - val_loss: 0.3096 - val_accuracy: 0.8590
Epoch 8/10

In [83]:
# Evaluate model performance on test set using precision, recall, and F1-score
y_true = []
y_pred = []

# Iterate through test dataset to get predictions
for images, labels in test_data:
    preds = model.predict(images)
    y_true.extend(labels)
    y_pred.extend(preds.round())
    if len(y_true) >= test_data.samples:
        break

# Compute and print classification metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA']))
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

1/1 [==============================] - 0s 174ms/step
              precision    recall  f1-score   support

      NORMAL       0.86      0.90      0.88       234
   PNEUMONIA       0.94      0.92      0.93       390

    accuracy                           0.91       624
   macro avg       0.90      0.91      0.90       624
weighted avg       0.91      0.91      0.91       624

Precision: 0.9370
Recall: 0.9154
F1-Score: 0.9261


In [84]:
#When medical imaging is the use case, high recall should be favoured. 
#Also, there is an overall balance in the model with a good F1-score.
# Save the trained model to a file for later reuse
model.save('pneumonia_cnn_model.keras')